Önálló Laboratórium
    YOLO8 Gyakorlás
        PASCAL VOC0712 Dataset

Show X image, so you can see how does the original segmentation bitmask looks like. //
X maszk megjelenítése, hogy az eredeti bitmaszkok is láthatóak legyenek

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

#Get the file names from SegmantationObject folder)
folder_path = 'C:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\SegmentationObject\\'
file_names = os.listdir(folder_path)
#Show x images from the dataset
counter = 0
for file_name in file_names:
    if counter >= 1:
        break

    image_path = os.path.join(folder_path, file_name)
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image at {image_path}")
        continue

    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.show()

    counter += 1

Determine the colors used in the images, so the edge and background colors can be excluded later. //
A P formátumú színek kódjának meghatározása pixel számmal, ennek segítségével később a háttér és a maszk széle kihagyható a címkegeneráláshoz.

In [2]:
from PIL import Image
import numpy as np
# Code to determine the P color of the background and the edges, so these can be excluded in the next step
def print_color_areas(image_path):
    # Load image in P format
    img = Image.open(image_path)
    img_array = np.array(img)

    # Get unique colors and their counts
    unique_colors, counts = np.unique(img_array, return_counts=True)

    # Print color numbers and their areas
    for color, count in zip(unique_colors, counts):
        print(f'Color number: {color}, Area: {count}')

print_color_areas("C:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\SegmentationObject\\2007_000032.png")
print_color_areas("C:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\SegmentationObject\\2007_000039.png")
#from the results we can see that the background is 0 and the edges are 255

Color number: 0, Area: 129545
Color number: 1, Area: 4083
Color number: 2, Area: 651
Color number: 3, Area: 417
Color number: 4, Area: 449
Color number: 255, Area: 5355
Color number: 0, Area: 154539
Color number: 1, Area: 30031
Color number: 255, Area: 2930


In [3]:
import os
import xml.etree.ElementTree as ET

#Get the classes from the dataset and print them
folder_path_xml = 'C:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\Annotations'
file_names = os.listdir(folder_path_xml)

classes = set()

for file_name in file_names:
    xml_path = os.path.join(folder_path_xml, file_name)
    tree = ET.parse(xml_path)
    root = tree.getroot()

    for obj in root.iter('object'):
        class_name = obj.find('name').text
        classes.add(class_name)

print(classes)

{'sofa', 'cat', 'motorbike', 'person', 'horse', 'car', 'train', 'bicycle', 'cow', 'boat', 'chair', 'diningtable', 'pottedplant', 'sheep', 'tvmonitor', 'dog', 'bird', 'bottle', 'bus', 'aeroplane'}


Defining a function to convert the multi color bitmasks to yolo segmentation format text files(class name, x1,y1,x2,y2...xn,yn) //
Címkegeneráláshoz szükséges function definiálása. A címkegenerálás többszínű maszkokból készít yolo szegmentációs formátumú szöveges fájlokat(osztály név, x1, y1, x2, y2 ...xn, yn)

In [4]:
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
from scipy import stats
import glob
import os
import skimage.measure
import cv2


def is_clockwise(contour):
    # calculate the signed area
    signed_area = 0.5 * np.sum((contour[:-1, 0] * contour[1:, 1]) - (contour[:-1, 1] * contour[1:, 0]))
    # If the signed area is negative, the points are in clockwise order
    return signed_area < 0
# Classes
class_names = {
    "sofa": 0,
    "train": 1,
    "bicycle": 2,
    "horse": 3,
    "motorbike": 4,
    "sheep": 5,
    "cat": 6,
    "bird": 7,
    "bottle": 8,
    "cow": 9,
    "bus": 10,
    "pottedplant": 11,
    "diningtable": 12,
    "person": 13,
    "tvmonitor": 14,
    "chair": 15,
    "dog": 16,
    "boat": 17,
    "car": 18,
    "aeroplane": 19
}


def generate_labels(xml_file, png_file, label_file):
    # Parse XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Load PNG mask in P format
    img = Image.open(png_file).convert('P')
    mask = np.array(img)

    # Get image dimensions for normalization
    width, height = img.size

    # Initialize color-object mapping and class name mapping
    color_object_mapping = {}
    class_name_mapping = {}

    # Get all objects
    objects = list(root.findall('object'))

    # For each object in the XML file
    for idx, obj in enumerate(objects):
        # Get class name and bounding box
        class_name = obj.find('name').text
        class_name_mapping[idx] = class_name 
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)

        # Extract region from bounding box coordinates
        region = mask[ymin:ymax, xmin:xmax]

        # Exclude edge color(255) and black(0)
        region = region[(region != 0) & (region != 255)]

        # For each unique color in the region
        for color_index in np.unique(region):
            # Map color index to list of objects
            color_object_mapping.setdefault(color_index, []).append(idx)

     # Open label file for writing
    with open(label_file, 'w') as f:
        # For each unique color index in mask
        for color_index in np.unique(mask.reshape(-1)):
            # If color index is mapped to an object
            if color_index in color_object_mapping:
                # Get object IDs
                obj_ids = color_object_mapping[color_index]

                # Find contours of this color index and add a border to avoid edge cases
                border_mask = cv2.copyMakeBorder(mask, 1, 1, 1, 1, cv2.BORDER_CONSTANT, value=0)
                contours = skimage.measure.find_contours(border_mask == color_index, 0.5)

                for contour in contours:
                    # Ensure contour is in clockwise order
                    if not is_clockwise(contour):
                        contour = contour[::-1]

                    # Check if contour size is less than 10
                    if contour.shape[0] < 10:
                        continue

                    # Calculate bounding box of contour, so it can be compared later to the bounding box from the xml files
                    min_y, min_x = np.min(contour, axis=0)
                    max_y, max_x = np.max(contour, axis=0)

                    # Initialize max intersection to a default value,it is used to find the most similar bounding box
                    max_intersection = 0

                    # Find object whose bounding box best matches the contour's bounding box
                    for obj_id in obj_ids:
                        obj = objects[obj_id]
                        bndbox = obj.find('bndbox')
                        xmin = int(bndbox.find('xmin').text)
                        ymin = int(bndbox.find('ymin').text)
                        xmax = int(bndbox.find('xmax').text)
                        ymax = int(bndbox.find('ymax').text)

                        # Calculate intersection of object's bounding box with contour's bounding box
                        intersection = max(0, min(xmax, max_x) - max(xmin, min_x)) * max(0, min(ymax, max_y) - max(ymin, min_y))

                        # Calculate area of contour's bounding box
                        contour_area = (max_x - min_x) * (max_y - min_y)

                        class_name = class_name_mapping[obj_id]  # assign class number, so if the if part doesn't run, there will still be an assigned class number
                        class_number = class_names[class_name]
                        # Initialize max_ratio, max_area and class_number to default values
                        max_ratio = 0
                        max_area = float('inf')

                        # Find object whose bounding box best matches the contour's bounding box
                        for obj_id in obj_ids:
                            obj = objects[obj_id]
                            bndbox = obj.find('bndbox')
                            xmin = int(bndbox.find('xmin').text)
                            ymin = int(bndbox.find('ymin').text)
                            xmax = int(bndbox.find('xmax').text)
                            ymax = int(bndbox.find('ymax').text)

                            # Calculate intersection of object's bounding box with contour's bounding box
                            intersection = max(0, min(xmax, max_x) - max(xmin, min_x)) * max(0, min(ymax, max_y) - max(ymin, min_y))

                            # Calculate area of contour's bounding box
                            contour_area = (max_x - min_x) * (max_y - min_y)

                            # Calculate area of object's bounding box
                            object_area = (xmax - xmin) * (ymax - ymin)

                            # Calculate ratio of intersection area to contour area
                            ratio = intersection / contour_area if contour_area > 0 else 0

                            # If ratio is larger than the current maximum ratio, or if ratio is equal to the maximum ratio and object area is smaller than the current maximum area, then update the maximum ratio, maximum area, and class name+number
                            if ratio > max_ratio or (ratio == max_ratio and object_area < max_area):
                                max_ratio = ratio
                                max_area = object_area
                                class_name = class_name_mapping[obj_id]  # Get class name from dictionary
                                class_number = class_names[class_name]

                    # Initialize a string to store all contours of an object
                    contour_str = ''
                    for y, x in contour:
                        # Normalize coordinates and remove the added border
                        contour_str += f' {(x-1)/width} {(y-1)/height}'

                    # Write class number and all contour coordinates to label file
                    f.write(f'{class_number}{contour_str}\n')

In [7]:
# Get all XML files in folder
xml_files = glob.glob('C:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\Annotations\\*.xml')
png_files_dir = 'C:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\SegmentationObject\\'
label_files_dir = 'C:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\IMGAnnotations\\'

# For each XML file
for xml_file in xml_files:
    # Extract the base name
    base_name = os.path.splitext(os.path.basename(xml_file))[0]
    
    # Generate PNG and TXT file paths
    png_file = os.path.join(png_files_dir, base_name + '.png')
    label_file = os.path.join(label_files_dir, base_name + '.txt')

    # Check if PNG file exists
    if not os.path.exists(png_file):
        continue

    # Generate labels
    generate_labels(xml_file, png_file, label_file)

Previous versions of the label generation, these generated mislabeled and/or mismasked labels //
Régebbi változata a címkegeneráló kódoknak, ezek a próbálkozások még több esetben rossz címkéket és/vagy rossz maszkokat generált.

In [ ]:
""" import cv2
import numpy as np
import os
import xml.etree.ElementTree as ET

folder_path_img = 'G=\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\SegmentationObject\\'
folder_path_xml = 'G=\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\Annotations'
file_names_xml = os.listdir(folder_path_xml)
file_names = os.listdir(folder_path_img)

classes = set()
for file_name in file_names_xml:
    xml_path = os.path.join(folder_path_xml, file_name)
    tree = ET.parse(xml_path)
    root = tree.getroot()

    for obj in root.iter('object'):
        class_name = obj.find('name').text
        classes.add(class_name)

class_to_index = {class_name: index for index, class_name in enumerate(classes)}

for file_name in file_names:
    image_path = os.path.join(folder_path_img, file_name)
    xml_path = os.path.join(folder_path_xml, file_name.replace('.png', '.xml'))
    image = cv2.imread(image_path)

    # Parse the corresponding XML file
    tree = ET.parse(xml_path)
    root = tree.getroot()

    polygons_dict = {}
    bounding_boxes = []
    for obj in root.iter('object'):
        class_name = obj.find('name').text
        class_index = class_to_index.get(class_name, -1)
        if class_index == -1:
            continue  # skip classes not in the mapping

        # Get the bounding box coordinates
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)
        bounding_boxes.append([xmin, ymin, xmax, ymax])

    # Group overlapping bounding boxes
    grouped_bounding_boxes, weights = cv2.groupRectangles(np.array(bounding_boxes).tolist(), 1, 0.2)
    # Sort the bounding boxes by area in descending order
    bounding_boxes = sorted(bounding_boxes, key=lambda bbox: (bbox[2] - bbox[0]) * (bbox[3] - bbox[1]))

    for index, bbox in enumerate(bounding_boxes):
        xmin, ymin, xmax, ymax = bbox
        # Extract the segment from the segmentation mask
        segment = image[ymin:ymax, xmin:xmax]

        # Find all unique colors in the image
        colors = np.unique(segment.reshape(-1, segment.shape[2]), axis=0)
        # Exclude black color and edge color
        colors = colors[np.all(colors != [0, 0, 0], axis=1)]
        colors = colors[np.all(colors != [224, 224, 192], axis=1)]

    for color in set(tuple(c) for c in colors):            # Create a mask for the object with the current color
            mask = np.all(image == color, axis=2)

            # Find the contours in the mask
            contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            for contour in contours:
                epsilon = 0.001 * cv2.arcLength(contour, True)
                approx = cv2.approxPolyDP(contour, epsilon, True)

                # Calculate the centroid of the points
                M = cv2.moments(approx)
                if M["m00"] != 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                else:
                    # Use the center of the bounding box as the default value
                    cX, cY = (xmin + xmax) // 2, (ymin + ymax) // 2

                # Sort the points in clockwise order
                sorted_contour = sorted(approx, key=lambda pt: np.arctan2(pt[0][1] - cY, pt[0][0] - cX))

                polygon = np.array(sorted_contour).reshape(-1, 2)

                # Use a tuple of class index, bounding box, and index as the key
                key = (class_index, (xmin, ymin, xmax, ymax), index)
                if key in polygons_dict:
                    if not any(np.array_equal(polygon, existing_polygon) for existing_polygon in polygons_dict[key]):
                            # Append the new polygon to the list of polygons for this key
                            polygons_dict[key].append(polygon)
                else:
                    # Add a new entry to the dictionary with a list containing the polygon
                    polygons_dict[key] = [polygon]

    height, width, _ = image.shape
    polygons_list = []
    for key, polygons in polygons_dict.items():
        class_index, _, _ = key  # Extract the class index from the key
        for polygon in polygons:
            polygons_list.append((class_index, polygon))

    txt_file_name = os.path.join('IMGAnnotations', file_name.replace('.png', '.txt'))
    with open(txt_file_name, 'w') as f:
        for class_index, polygon in polygons_list:
            if len(polygon) >= 2:  # Ensure there are at least two points in the polygon
                points = polygon.flatten()
                f.write(f"{class_index} {' '.join(map(str, points))}\n") """

In [8]:
"""
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
from scipy import stats
import glob
import os
import skimage.measure
import cv2


def is_clockwise(contour):
    # Use the Shoelace formula to calculate the signed area
    signed_area = 0.5 * np.sum((contour[:-1, 0] * contour[1:, 1]) - (contour[:-1, 1] * contour[1:, 0]))
    # If the signed area is negative, the points are in clockwise order
    return signed_area < 0
# Classes
class_names = {
    "sofa": 0,
    "train": 1,
    "bicycle": 2,
    "horse": 3,
    "motorbike": 4,
    "sheep": 5,
    "cat": 6,
    "bird": 7,
    "bottle": 8,
    "cow": 9,
    "bus": 10,
    "pottedplant": 11,
    "diningtable": 12,
    "person": 13,
    "tvmonitor": 14,
    "chair": 15,
    "dog": 16,
    "boat": 17,
    "car": 18,
    "aeroplane": 19
}


def generate_labels(xml_file, png_file, label_file):
    # Parse XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Load PNG mask in P format
    img = Image.open(png_file).convert('P')
    mask = np.array(img)

    # Get image dimensions for normalization
    width, height = img.size

    # Initialize color-object mapping
    color_object_mapping = {}

    # Get all objects
    objects = list(root.findall('object'))

    # For each object in the XML file
    for idx, obj in enumerate(objects):
        # Get class name and bounding box
        class_name = obj.find('name').text + str(idx)  # Append index to class name
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)

        # Extract region from mask
        region = mask[ymin:ymax, xmin:xmax]

        # Exclude edge color(255) and black(0)
        region = region[(region != 0) & (region != 255)]

        # For each unique color in the region
        for color_index in np.unique(region):
            # Map color index to list of objects
            color_object_mapping.setdefault(color_index, []).append(obj)
    # Open label file for writing
    with open(label_file, 'w') as f:
        # For each unique color index in mask
        for color_index in np.unique(mask.reshape(-1)):
            # If color index is mapped to an object
            if color_index in color_object_mapping:
                # Get object and class number
                objs = color_object_mapping[color_index]
                if len(objs) > 1:
                    obj = min(objs, key=lambda o: (int(o.find('bndbox').find('xmax').text) - int(o.find('bndbox').find('xmin').text)) * (int(o.find('bndbox').find('ymax').text) - int(o.find('bndbox').find('ymin').text)))
                else:
                    obj = objs[0]
                class_name = obj.find('name').text
                class_number = class_names[class_name]

                # Find contours of this color index
                border_mask = cv2.copyMakeBorder(mask, 1, 1, 1, 1, cv2.BORDER_CONSTANT, value=0)
                contours = skimage.measure.find_contours(border_mask == color_index, 0.5)

                for contour in contours:
                    # Ensure contour is in clockwise order
                    if not is_clockwise(contour):
                        contour = contour[::-1]

                    # Initialize a string to store all contours of an object
                    contour_str = ''
                    for y, x in contour:
                        # Normalize and round coordinates to 5decimal places
                        x_rounded = (x-1)/width
                        y_rounded = (y-1)/height
                        contour_str += f' {x_rounded} {y_rounded}'

                    # Write class number and all contour coordinates to label file
                    f.write(f'{class_number}{contour_str}\n')
# Get all XML files in folder
xml_files = glob.glob('G:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\Annotations\\*.xml')
png_files_dir = 'G:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\SegmentationObject\\'
label_files_dir = 'G:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\IMGAnnotations\\'

# For each XML file
for xml_file in xml_files:
    # Extract the base name
    base_name = os.path.splitext(os.path.basename(xml_file))[0]
    
    # Generate PNG and TXT file paths
    png_file = os.path.join(png_files_dir, base_name + '.png')
    label_file = os.path.join(label_files_dir, base_name + '.txt')

    # Check if PNG file exists
    if not os.path.exists(png_file):
        continue

    # Generate labels
    generate_labels(xml_file, png_file, label_file) """

' import xml.etree.ElementTree as ET\nfrom PIL import Image\nimport numpy as np\nfrom scipy import stats\nimport glob\nimport os\nimport skimage.measure\nimport cv2\n\n\ndef is_clockwise(contour):\n    # Use the Shoelace formula to calculate the signed area\n    signed_area = 0.5 * np.sum((contour[:-1, 0] * contour[1:, 1]) - (contour[:-1, 1] * contour[1:, 0]))\n    # If the signed area is negative, the points are in clockwise order\n    return signed_area < 0\n# Classes\nclass_names = {\n    "sofa": 0,\n    "train": 1,\n    "bicycle": 2,\n    "horse": 3,\n    "motorbike": 4,\n    "sheep": 5,\n    "cat": 6,\n    "bird": 7,\n    "bottle": 8,\n    "cow": 9,\n    "bus": 10,\n    "pottedplant": 11,\n    "diningtable": 12,\n    "person": 13,\n    "tvmonitor": 14,\n    "chair": 15,\n    "dog": 16,\n    "boat": 17,\n    "car": 18,\n    "aeroplane": 19\n}\n\n\ndef generate_labels(xml_file, png_file, label_file):\n    # Parse XML file\n    tree = ET.parse(xml_file)\n    root = tree.getroot()\n\n

In [9]:
""" 
def generate_labels(xml_file, png_file, label_file):
    # Parse XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Load PNG mask in P format
    img = Image.open(png_file).convert('P')
    mask = np.array(img)

    # Get image dimensions for normalization
    width, height = img.size

    # Initialize class-color mapping
    class_color_mapping = {}

    # Get all objects
    objects = list(root.findall('object'))

    # Calculate area of each bounding box and sort objects by area
    objects.sort(key=lambda obj: (int(obj.find('bndbox/xmax').text) - int(obj.find('bndbox/xmin').text)) * (int(obj.find('bndbox/ymax').text) - int(obj.find('bndbox/ymin').text)))
    
    # For each object in the XML file
    for obj in objects:
        # Get class name and bounding box
        class_name = obj.find('name').text
        class_number = class_names[class_name]  # Get class number from dictionary
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)

        # Extract region from mask
        region = mask[ymin:ymax, xmin:xmax]

        # Exclude edge color and black
        region = region[(region != 0) & (region != 255)]

        # For each unique color in the region
        for color_index in np.unique(region):
            # If color index is not already mapped to a class
            if color_index not in class_color_mapping:
                # Map color index to class number
                class_color_mapping[color_index] = class_number

    # Open label file for writing
    with open(label_file, 'w') as f:
        # For each unique color index in mask
        for color_index in np.unique(mask.reshape(-1)):
            # If color index is mapped to a class
            if color_index in class_color_mapping:
                # Find contours of this color index
                contours = skimage.measure.find_contours(mask == color_index, 0.5)

                # Write class number and contour coordinates to label file
                f.write(f'{class_color_mapping[color_index]}')
                for contour in contours:
                    # Ensure contour is in clockwise order
                    if not is_clockwise(contour):
                        contour = contour[::-1]

                    for y, x in contour:
                        # Normalize and round coordinates to 5 decimal places
                        x_rounded = round(x/width, 5)
                        y_rounded = round(y/height, 5)
                        f.write(f' {x_rounded} {y_rounded}')
                f.write('\n') """
""" def generate_labels(xml_file, png_file, label_file):
    # Parse XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Load PNG mask in P format
    img = Image.open(png_file).convert('P')
    mask = np.array(img)

    # Get image dimensions for normalization
    width, height = img.size

    # Initialize class-color mapping
    class_color_mapping = {}

    # Get all objects
    objects = list(root.findall('object'))

    # Calculate area of each bounding box and sort objects by area
    objects.sort(key=lambda obj: (int(obj.find('bndbox/xmax').text) - int(obj.find('bndbox/xmin').text)) * (int(obj.find('bndbox/ymax').text) - int(obj.find('bndbox/ymin').text)))

    # For each object in the XML file
    for obj in objects:
        # Get class name and bounding box
        class_name = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)

        # Extract region from mask
        region = mask[ymin:ymax, xmin:xmax]

        # Exclude edge color and black
        region = region[(region != 0) & (region != 224)]

        # For each unique color in the region
        for color_index in np.unique(region):
            # Map color index to class name
            class_color_mapping[color_index] = class_name

    # Open label file for writing
    with open(label_file, 'w') as f:
        # For each unique color index in mask
        for color_index in np.unique(mask.reshape(-1)):
            # If color index is mapped to a class
            if color_index in class_color_mapping:
                # Find pixels that have this color index
                pixels = np.argwhere(mask == color_index)

                # Write class name and pixel coordinates to label file
                f.write(f'{class_color_mapping[color_index]}')
                for y, x in pixels:
                    f.write(f' {x/width} {y/height}')
                f.write('\n') """
""" # Specify the paths to your XML and PNG files
xml_file = 'G:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\Annotations\\2007_000032.xml'
png_file = 'G:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\SegmentationClass\\2007_000032.png'

# Specify the path to the output label file
label_file = 'G:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\2007.txt'

# Check if PNG file exists
if not os.path.exists(png_file):
    print(f"PNG file {png_file} does not exist.")
else:
    # Generate labels
    generate_labels(xml_file, png_file, label_file)
    print(f"Labels generated and saved to {label_file}.") """

' # Specify the paths to your XML and PNG files\nxml_file = \'G:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\Annotations\\2007_000032.xml\'\npng_file = \'G:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\SegmentationClass\\2007_000032.png\'\n\n# Specify the path to the output label file\nlabel_file = \'G:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\2007.txt\'\n\n# Check if PNG file exists\nif not os.path.exists(png_file):\n    print(f"PNG file {png_file} does not exist.")\nelse:\n    # Generate labels\n    generate_labels(xml_file, png_file, label_file)\n    print(f"Labels generated and saved to {label_file}.") '

In [10]:
"""
#Modified version of the previous code, because the previous one had multiple objects per class, and YOLOV8 deleted the multiple objects #Not true, there were  overlapping boxes, that caused the problem
import cv2
import numpy as np
import os
import xml.etree.ElementTree as ET

# Get the classes from the dataset for further use
folder_path_img = 'G:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\SegmentationObject\\'
folder_path_xml = 'G:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\Annotations'
file_names_xml = os.listdir(folder_path_xml)
file_names = os.listdir(folder_path_img)

classes = set()
for file_name in file_names_xml:
    xml_path = os.path.join(folder_path_xml, file_name)
    tree = ET.parse(xml_path)
    root = tree.getroot()

    for obj in root.iter('object'):
        class_name = obj.find('name').text
        classes.add(class_name)

class_to_index = {class_name: index for index, class_name in enumerate(classes)}

for file_name in file_names:
    image_path = os.path.join(folder_path_img, file_name)
    xml_path = os.path.join(folder_path_xml, file_name.replace('.png', '.xml'))
    image = cv2.imread(image_path)

    # Parse the corresponding XML file
    tree = ET.parse(xml_path)
    root = tree.getroot()

    polygons_per_class = {class_index: [] for class_index in class_to_index.values()}
    for obj in root.iter('object'):
        class_name = obj.find('name').text
        class_index = class_to_index.get(class_name, -1)
        if class_index == -1:
            continue  # skip classes not in the mapping

        # Get the bounding box coordinates
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)

        # Extract the segment from the segmentation mask
        segment = image[ymin:ymax, xmin:xmax]

        # Find the unique color in the segment
        colors = np.unique(segment.reshape(-1, segment.shape[2]), axis=0)

        # Exclude black color
        colors = colors[np.any(colors != [0, 0, 0], axis=1)]

        # Assume the most common color in the segment is the object color
        color = colors[0] if len(colors) > 0 else [0, 0, 0]

        # Create a mask for the object within the bounding box
        mask = np.all(segment == color, axis=2)

        # Find the contours in the mask
        contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            # Adjust the contour points to be relative to the entire image, not just the segment
            contour += [xmin, ymin]
            polygon = contour.reshape(-1, 2)
            polygons_per_class[class_index].append(polygon)

    height, width, _ = image.shape
    normalized_polygons = []
    for class_index, polygons in polygons_per_class.items():
        # Skip if there are no polygons for this class
        if not polygons:
            continue

        # Merge all polygons of the same class into a single polygon
        merged_polygon = np.concatenate(polygons)
        normalized_polygon = merged_polygon / [width, height]
        normalized_polygons.append((class_index, normalized_polygon))

    txt_file_name = os.path.join('IMGAnnotations', file_name.replace('.png', '.txt'))
    with open(txt_file_name, 'w') as f:
        for class_index, polygon in normalized_polygons:
            points = polygon.flatten()
            f.write(f"{class_index} {' '.join(map(str, points))}\n")
"""

'\n#Modified version of the previous code, because the previous one had multiple objects per class, and YOLOV8 deleted the multiple objects #Not true, there were  overlapping boxes, that caused the problem\nimport cv2\nimport numpy as np\nimport os\nimport xml.etree.ElementTree as ET\n\n# Get the classes from the dataset for further use\nfolder_path_img = \'G:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\SegmentationObject\\\'\nfolder_path_xml = \'G:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\Annotations\'\nfile_names_xml = os.listdir(folder_path_xml)\nfile_names = os.listdir(folder_path_img)\n\nclasses = set()\nfor file_name in file_names_xml:\n    xml_path = os.path.join(folder_path_xml, file_name)\n    tree = ET.parse(xml_path)\n    root = tree.getroot()\n\n    for obj in root.iter(\'object\'):\n        class_name = obj.find(\'name\').text\n        classes.add(class_name)\n\nclass_to_index = {class_name: index for index, class_name in enumerate(classes)}\n\nfor file_name in file_names:\n 

Display a few randomly selected image to verify if the previous annotation conversion is correct or not. //
Pár random kiválasztott kép megjelenítése a saját címkékből készített maszkokkal, így ellenőrizve a generált címkéket.

In [ ]:
import os
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Define a dictionary to map class names to indices
index_to_class_dict = {
    0: 'sofa',
    1: 'train',
    2: 'bicycle',
    3: 'horse',
    4: 'motorbike',
    5: 'sheep',
    6: 'cat',
    7: 'bird',
    8: 'bottle',
    9: 'cow',
    10: 'bus',
    11: 'pottedplant',
    12: 'diningtable',
    13: 'person',
    14: 'tvmonitor',
    15: 'chair',
    16: 'dog',
    17: 'boat',
    18: 'car',
    19: 'aeroplane'
}
# Directory containing the images
image_directory = 'C:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\JPEGImages'

# Directory containing the text files
text_directory = 'C:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\IMGAnnotations'

# Get all text files in the text directory
text_files = [f for f in os.listdir(text_directory) if f.endswith('.txt')]

# Randomly select 5 text files
selected_files = random.sample(text_files, 5)

# For each selected file
for file in selected_files:
    # Read the image
    image = cv2.imread(os.path.join(image_directory, os.path.splitext(file)[0] + '.jpg'))

    # Create an empty list to store the masks and class names
    masks = []
    class_names = []

    # Open the corresponding text file
    with open(os.path.join(text_directory, file)) as f:
        # For each line in the file
        for line in f:
            # Split the line into class index and coordinates
            parts = line.split()
            class_index = int(parts[0])  # Convert the class index to an integer
            class_name = index_to_class_dict[class_index]  # Use the index-to-class dictionary to get the class name
            coords = list(map(float, parts[1:]))

            # Denormalize the coordinates
            coords_denorm = [(int(coord[0] * image.shape[1]), int(coord[1] * image.shape[0])) for coord in np.array(coords).reshape(-1, 2)]

            # Create a separate mask for this object
            object_mask = np.zeros_like(image)

            # Draw the polygon on the object mask
            cv2.fillPoly(object_mask, [np.array(coords_denorm, dtype=np.int32)], (255, 255, 255))

            # Add the object mask and class name to the lists
            masks.append(object_mask)
            class_names.append(class_name)

    # Display the image and each mask
    plt.figure(figsize=(10, 5 * len(masks)))
    plt.subplot(len(masks) + 1, 2, 1)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(f'Image - {file}')  # Display the file name along with the image
    for i, (mask, class_name) in enumerate(zip(masks, class_names), start=1):
        plt.subplot(len(masks) + 1, 2, 2 * i + 1)
        plt.imshow(mask, cmap='gray')
        plt.title(f'Mask - {class_name} - {file}')  # Display the file name along with the class name and mask
    plt.show()

Copy the images and the generated labels to train test and validation directories for the training session. //
A generált fájlok és az eredeti képek yolonak megfelelő mappákba másolása.

In [13]:
import os
import shutil
import numpy as np

# Define the directories
base_dir_images = 'C:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\JPEGImages'
base_dir_labels = 'C:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\IMGAnnotations'
train_dir_images = 'dataset/train/images'
train_dir_labels = 'dataset/train/labels'
valid_dir_images = 'dataset/valid/images'
valid_dir_labels = 'dataset/valid/labels'
test_dir_images = 'dataset/test/images'
test_dir_labels = 'dataset/test/labels'

# Create the directories
os.makedirs(train_dir_images, exist_ok=True)
os.makedirs(train_dir_labels, exist_ok=True)
os.makedirs(valid_dir_images, exist_ok=True)
os.makedirs(valid_dir_labels, exist_ok=True)
os.makedirs(test_dir_images, exist_ok=True)
os.makedirs(test_dir_labels, exist_ok=True)

# Get all the labels in the base directory
# Checking labels, because the images not necessarily have labels
labels = [f for f in os.listdir(base_dir_labels) if f.endswith('.txt')]

# Sort the labels
labels.sort()

# Create a list of corresponding images
images = [f.replace('.txt', '.jpg') for f in labels]

# Shuffle the indices
indices = np.arange(len(labels))
np.random.shuffle(indices)

# Split the indices into 70% train, 15% validation, and 15% test
train_indices, validate_indices, test_indices = np.split(indices, [int(.7*len(indices)), int(.85*len(indices))])

# Copy the images and labels into the right directories
for idx in train_indices:
    shutil.copy(os.path.join(base_dir_images, images[idx]), os.path.join(train_dir_images, images[idx]))
    shutil.copy(os.path.join(base_dir_labels, labels[idx]), os.path.join(train_dir_labels, labels[idx]))
for idx in validate_indices:
    shutil.copy(os.path.join(base_dir_images, images[idx]), os.path.join(valid_dir_images, images[idx]))
    shutil.copy(os.path.join(base_dir_labels, labels[idx]), os.path.join(valid_dir_labels, labels[idx]))
for idx in test_indices:
    shutil.copy(os.path.join(base_dir_images, images[idx]), os.path.join(test_dir_images, images[idx]))
    shutil.copy(os.path.join(base_dir_labels, labels[idx]), os.path.join(test_dir_labels, labels[idx]))

The used YAML file for the yolo training //
A yolo tanításhoz használt YAML fájl :

In [14]:
#YAML file:
"""
path: C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset  # dataset root dir
train: C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\train\images  # train images relative path
val: C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\valid\images  # val images relative path
test: C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\test\images  # test images relative path

# Classes
names:
  0: sofa
  1: train
  2: bicycle
  3: horse
  4: motorbike
  5: sheep
  6: cat
  7: bird
  8: bottle
  9: cow
  10: bus
  11: pottedplant
  12: diningtable
  13: person
  14: tvmonitor
  15: chair
  16: dog
  17: boat
  18: car
  19: aeroplane
  """

'\npath: dataset  # dataset root dir\ntrain: train/images  # train images relative path\nval: valid/images  # val images relative path\ntest: test/images  # test images relative path\n\n# Classes\nnames:\n  0: sofa\n  1: train\n  2: bicycle\n  3: horse\n  4: motorbike\n  5: sheep\n  6: cat\n  7: bird\n  8: bottle\n  9: cow\n  10: bus\n  11: pottedplant\n  12: diningtable\n  13: person\n  14: tvmonitor\n  15: chair\n  16: dog\n  17: boat\n  18: car\n  19: aeroplane\n  '

Checking if the current system has CUDA compatible GPU and tune the model hyperparameters with few epoch runs //
CUDA verzió és elérhetőség ellenőrzése, majd hiperparaméterek hangolása a saját adathalmazon.

In [1]:
#CUDA check
import torch
print("cuDNN version:", torch.backends.cudnn.version())
torch.cuda.is_available()


cuDNN version: 8801


True

In [1]:
#Tuning the hyperparameters on a shorter run
# Documentation and examples from : https://docs.ultralytics.com/guides/hyperparameter-tuning/#usage-example
from ultralytics import YOLO

model = YOLO('yolov8n-seg.pt')
model.tune(data='dataset/dataset.yaml',optimizer='AdamW', epochs=10, iterations=50,imgsz=500, batch=-1,workers=4)

Tuner: Initialized Tuner instance with 'tune_dir=runs\segment\tune7'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/50 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'mosaic': 1.0, 'mixup': 0.0, 'copy_paste': 0.0}
Saved runs\segment\tune7\tune_scatter_plots.png
Saved runs\segment\tune7\tune_fitness.png

Tuner: 1/50 iterations complete ✅ (2137.88s)
Tuner: Results saved to runs\segment\tune7
Tuner: Best fitness=0.5238 observed at iteration 1
Tuner: Best fitness metrics are {'metrics/precision(B)': 0.69561, 'metrics/recall(B)': 0.31793, 'metrics/mAP50(B)': 0.35635, 'metrics/mAP50-95(B)': 0.26826, 'metrics/precision(M)': 0.68679, 'metrics/reca

Start the training,first cell contains the training with the default hyperparameters(this run was before the tuning), the next cell trains with the tuned hyperparameters //
Tanítás először még az alap paraméterekkel, majd a hangolt paraméterekkel.(az első tanítás még az előző hangolás előtt történt időben, de ha már végigfutott benthagytam.)

In [1]:
#Training
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n-seg.pt')  # load a pretrained model
#model = YOLO('G:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\runs\\segment\\train\\weights\\last.pt')# load the last training model
# Train the model with default hyperparameters(this was the first training sessioon, i used the tuned hyperparameters in the second training in the next cell)
results = model.train(data='dataset/dataset.yaml', epochs=200, imgsz=500, batch=16, workers=8)

New https://pypi.org/project/ultralytics/8.1.32 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.27 🚀 Python-3.11.8 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 8192MiB)
engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=dataset/dataset.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=768, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None,

train: Scanning C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\train\labels.cache... 2646 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2646/2646 [00:00<?, ?it/s]

train: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\train\images\2009_004140.jpg: 1 duplicate labels removed
train: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\train\images\2010_000187.jpg: 1 duplicate labels removed
train: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\train\images\2010_001748.jpg: 1 duplicate labels removed
train: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\train\images\2010_001773.jpg: 1 duplicate labels removed
train: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\train\images\2010_002868.jpg: 9 duplicate labels removed
train: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\train\images\2010_005284.jpg: 1 duplicate labels removed



val: Scanning C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\valid\labels.cache... 807 images, 0 backgrounds, 0 corrupt: 100%|██████████| 807/807 [00:00<?, ?it/s]

val: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\valid\images\2007_009295.jpg: 1 duplicate labels removed
val: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\valid\images\2008_007737.jpg: 3 duplicate labels removed
val: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\valid\images\2009_001514.jpg: 1 duplicate labels removed
val: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\valid\images\2009_004140.jpg: 1 duplicate labels removed
val: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\valid\images\2010_000187.jpg: 1 duplicate labels removed
val: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\valid\images\2010_000748.jpg: 1 duplicate labels removed
val: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\valid\images\2010_002868.jpg: 9 duplicate labels removed


...

    200/200      6.23G     0.8617      1.914     0.6993      1.012        115        768: 100%|██████████| 166/166 [00:33<00:00,  4.93it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:06<00:00,  4.28it/s]

                   all        807       7092      0.859      0.443      0.492      0.397      0.839      0.431      0.476      0.339



200 epochs completed in 2.329 hours.
Optimizer stripped from runs\segment\train4\weights\last.pt, 6.8MB
Optimizer stripped from runs\segment\train4\weights\best.pt, 6.8MB

Validating runs\segment\train4\weights\best.pt...
Ultralytics YOLOv8.1.27 🚀 Python-3.11.8 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 8192MiB)
YOLOv8n-seg summary (fused): 195 layers, 3261964 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:05<00:00,  4.96it/s]


                   all        807       7092      0.845      0.448      0.496      0.401      0.826      0.433      0.478      0.341
                  sofa        807        112      0.821      0.705      0.727      0.643      0.814      0.696      0.703      0.596
                 train        807        163      0.842      0.459      0.513      0.416       0.84      0.448      0.495      0.364
               bicycle        807        731      0.745      0.185      0.221      0.152      0.614      0.145      0.149     0.0629
                 horse        807        226      0.828      0.489      0.513      0.404      0.828      0.478      0.522      0.342
             motorbike        807        401      0.837      0.327      0.407      0.292      0.817      0.311      0.387      0.209
                 sheep        807        217      0.763      0.567      0.603      0.489      0.758      0.548      0.591      0.441
                   cat        807        185      0.904      0.465   

In [2]:
#Best hyperparameters from the tune session:
""" lr0: 0.0102
lrf: 0.00387
momentum: 0.81859
weight_decay: 0.00068
warmup_epochs: 3.46434
warmup_momentum: 0.89747
box: 8.86819
cls: 0.80673
dfl: 1.31027
hsv_h: 0.01541
hsv_s: 0.74576
hsv_v: 0.3294
degrees: 0.0
translate: 0.10281
scale: 0.19924
shear: 0.0
perspective: 0.0
flipud: 0.0
fliplr: 0.39194
mosaic: 0.68242
mixup: 0.0
copy_paste: 0.0 """

#Training
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n-seg.pt')  # load a pretrained model
#model = YOLO('G:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\runs\\segment\\train\\weights\\last.pt')# load the last training model
# Train the model with the tuned hyperparameters
results = model.train(data='dataset/dataset.yaml', epochs=200, imgsz=500, batch=16, workers=8, optimizer='AdamW', patience=30, lr0= 0.0102, lrf= 0.00387, momentum= 0.81859, weight_decay= 0.00068, warmup_epochs= 3.46434, warmup_momentum= 0.89747, box= 8.86819, cls= 0.80673, dfl= 1.31027, hsv_h= 0.01541, hsv_s= 0.74576, hsv_v= 0.3294, degrees= 0.0, translate= 0.10281, scale= 0.19924, shear= 0.0, perspective= 0.0, flipud= 0.0, fliplr= 0.39194, mosaic= 0.68242, mixup= 0.0, copy_paste= 0.0)

New https://pypi.org/project/ultralytics/8.1.34 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.27 🚀 Python-3.11.8 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 8192MiB)
engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=dataset/dataset.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=768, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None,

train: Scanning C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\train\labels.cache... 2646 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2646/2646 [00:00<?, ?it/s]

train: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\train\images\2009_004140.jpg: 1 duplicate labels removed
train: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\train\images\2010_000187.jpg: 1 duplicate labels removed
train: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\train\images\2010_001748.jpg: 1 duplicate labels removed
train: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\train\images\2010_001773.jpg: 1 duplicate labels removed
train: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\train\images\2010_002868.jpg: 9 duplicate labels removed
train: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\train\images\2010_005284.jpg: 1 duplicate labels removed



val: Scanning C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\valid\labels.cache... 807 images, 0 backgrounds, 0 corrupt: 100%|██████████| 807/807 [00:00<?, ?it/s]

val: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\valid\images\2007_009295.jpg: 1 duplicate labels removed
val: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\valid\images\2008_007737.jpg: 3 duplicate labels removed
val: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\valid\images\2009_001514.jpg: 1 duplicate labels removed
val: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\valid\images\2009_004140.jpg: 1 duplicate labels removed
val: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\valid\images\2010_000187.jpg: 1 duplicate labels removed
val: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\valid\images\2010_000748.jpg: 1 duplicate labels removed
val: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\valid\images\2010_002868.jpg: 9 duplicate labels removed


...

    200/200      6.06G     0.8755      2.023     0.8697     0.8233        118        768: 100%|██████████| 166/166 [00:37<00:00,  4.46it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:07<00:00,  3.27it/s]

                   all        807       7092      0.874      0.472      0.524      0.433      0.864      0.459      0.511      0.366



200 epochs completed in 2.611 hours.
Optimizer stripped from runs\segment\train2\weights\last.pt, 6.8MB
Optimizer stripped from runs\segment\train2\weights\best.pt, 6.8MB

Validating runs\segment\train2\weights\best.pt...
Ultralytics YOLOv8.1.27 🚀 Python-3.11.8 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 8192MiB)
YOLOv8n-seg summary (fused): 195 layers, 3261964 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:05<00:00,  4.40it/s]


                   all        807       7092      0.887      0.469      0.524      0.434      0.868      0.458      0.511      0.366
                  sofa        807        112      0.887        0.7      0.745       0.69      0.864      0.681      0.724      0.616
                 train        807        163      0.927      0.509      0.564      0.453      0.928      0.509      0.567      0.409
               bicycle        807        731      0.822      0.211      0.258      0.177      0.701      0.176        0.2     0.0881
                 horse        807        226      0.864        0.5      0.547      0.446      0.877      0.504      0.547      0.371
             motorbike        807        401      0.814      0.382      0.442      0.322      0.806      0.369      0.435      0.235
                 sheep        807        217      0.848      0.571      0.644      0.531      0.842      0.567       0.64      0.468
                   cat        807        185      0.952      0.486   

In [1]:
#Continue training, because the previous training results looked like it was still learning
from ultralytics import YOLO

# Load a model
#model = YOLO('yolov8n-seg.pt')  # load a pretrained model
model = YOLO('C:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\runs\\segment\\train3\\weights\\last.pt')# load the last training model
# Train the model with the tuned hyperparameters
results = model.train(data='dataset/dataset.yaml', epochs=500,cache=True,imgsz=500, batch=-1, workers=10, optimizer='AdamW', patience=30, lr0= 0.0102, lrf= 0.00387, momentum= 0.81859, weight_decay= 0.00068, warmup_epochs= 3.46434, warmup_momentum= 0.89747, box= 8.86819, cls= 0.80673, dfl= 1.31027, hsv_h= 0.01541, hsv_s= 0.74576, hsv_v= 0.3294, degrees= 0.0, translate= 0.10281, scale= 0.19924, shear= 0.0, perspective= 0.0, flipud= 0.0, fliplr= 0.39194, mosaic= 0.68242, mixup= 0.0, copy_paste= 0.0)

New https://pypi.org/project/ultralytics/8.1.34 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.27 🚀 Python-3.11.8 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 8192MiB)
engine\trainer: task=segment, mode=train, model=C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\runs\segment\train3\weights\last.pt, data=dataset/dataset.yaml, epochs=500, time=None, patience=30, batch=-1, imgsz=500, save=True, save_period=-1, cache=True, device=None, workers=10, project=None, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnost

train: Scanning C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\train\labels.cache... 2646 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2646/2646 [00:00<?, ?it/s]

train: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\train\images\2009_004140.jpg: 1 duplicate labels removed
train: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\train\images\2010_000187.jpg: 1 duplicate labels removed
train: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\train\images\2010_001748.jpg: 1 duplicate labels removed
train: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\train\images\2010_001773.jpg: 1 duplicate labels removed
train: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\train\images\2010_002868.jpg: 9 duplicate labels removed
train: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\train\images\2010_005284.jpg: 1 duplicate labels removed



train: Caching images (1.4GB True): 100%|██████████| 2646/2646 [00:00<00:00, 3829.23it/s]
val: Scanning C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\valid\labels.cache... 807 images, 0 backgrounds, 0 corrupt: 100%|██████████| 807/807 [00:00<?, ?it/s]

val: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\valid\images\2007_009295.jpg: 1 duplicate labels removed
val: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\valid\images\2008_007737.jpg: 3 duplicate labels removed
val: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\valid\images\2009_001514.jpg: 1 duplicate labels removed
val: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\valid\images\2009_004140.jpg: 1 duplicate labels removed
val: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\valid\images\2010_000187.jpg: 1 duplicate labels removed
val: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\valid\images\2010_000748.jpg: 1 duplicate labels removed
val: WARNING ⚠️ C:\Onlab\VOC_dataset\VOCdevkit\VOC2012\dataset\valid\images\2010_002868.jpg: 9 duplicate labels removed


...

    154/500      5.08G      1.104      2.526      1.486     0.9028        118        512: 100%|██████████| 95/95 [00:16<00:00,  5.73it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:04<00:00,  3.48it/s]

                   all        807       7092      0.833      0.409      0.451      0.361      0.782       0.36      0.384      0.219


Stopping training early as no improvement observed in last 30 epochs. Best results observed at epoch 124, best model saved as best.pt.
To update EarlyStopping(patience=30) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

154 epochs completed in 0.921 hours.
Optimizer stripped from runs\segment\train\weights\last.pt, 6.8MB
Optimizer stripped from runs\segment\train\weights\best.pt, 6.8MB

Validating runs\segment\train\weights\best.pt...
Ultralytics YOLOv8.1.27 🚀 Python-3.11.8 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 8192MiB)
YOLOv8n-seg summary (fused): 195 layers, 3261964 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:04<00:00,  3.45it/s]


                   all        807       7092      0.813      0.399      0.441       0.35      0.803      0.375      0.413      0.266
                  sofa        807        112       0.84      0.659      0.699      0.615      0.837      0.643       0.67      0.471
                 train        807        163      0.817       0.46      0.487      0.351      0.827      0.438      0.468      0.301
               bicycle        807        731      0.788      0.156      0.196      0.132      0.567      0.103     0.0973     0.0396
                 horse        807        226      0.783      0.425      0.455      0.353      0.804      0.407      0.446      0.252
             motorbike        807        401      0.779      0.263      0.323      0.228      0.799      0.247      0.292      0.145
                 sheep        807        217      0.796      0.512      0.551      0.426        0.8      0.499      0.552      0.369
                   cat        807        185       0.89      0.432   

Use the trained models to segment the images in the test folder, then displaying them next to each other. //
A tanítás utáni modellek használata a test mappában található képek szegmentálásához, majd ezek megjelenítése egymás mellett.

In [8]:
#Code from ultralytics example+slight modification
from ultralytics import YOLO
from PIL import Image
import os

# List of models
models = ['trainAdeamWopt+patience30', 'trainAdamWopt', 'trainautoopt', 'traindefhyppar']

source = 'c:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\test'

# Iterate through each model
for model_name in models:
    # Load a model
    model = YOLO(f'c:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012\\runs\\segment\\{model_name}\\weights\\best.pt')# load last training model
    
    # Run batched inference on a list of images
    results = model(source, stream=True)  # return a list of Results objects

    for i, r in enumerate(results):
        # Plot results image
        im_bgr = r.plot()  # BGR-order numpy array
        im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image
        
        # Create the directory if it doesn't exist
        os.makedirs(model_name, exist_ok=True)
        
        # Save results to disk
        r.save(filename=f'{model_name}\\results{i}.jpg')


image 1/22 c:\Onlab\VOC_dataset\VOCdevkit\VOC2012\test\2007_000027.jpg: 512x512 1 person, 4.0ms
image 2/22 c:\Onlab\VOC_dataset\VOCdevkit\VOC2012\test\2007_000032.jpg: 288x512 4 persons, 3 aeroplanes, 4.0ms
image 3/22 c:\Onlab\VOC_dataset\VOCdevkit\VOC2012\test\2007_000033.jpg: 384x512 8 aeroplanes, 4.0ms
image 4/22 c:\Onlab\VOC_dataset\VOCdevkit\VOC2012\test\2007_000039.jpg: 384x512 1 tvmonitor, 4.0ms
image 5/22 c:\Onlab\VOC_dataset\VOCdevkit\VOC2012\test\2007_000042.jpg: 352x512 6 trains, 6.3ms
image 6/22 c:\Onlab\VOC_dataset\VOCdevkit\VOC2012\test\2007_000061.jpg: 352x512 2 boats, 4.0ms
image 7/22 c:\Onlab\VOC_dataset\VOCdevkit\VOC2012\test\2007_000063.jpg: 384x512 2 chairs, 1 dog, 7.0ms
image 8/22 c:\Onlab\VOC_dataset\VOCdevkit\VOC2012\test\2007_000068.jpg: 384x512 1 bird, 5.0ms
image 9/22 c:\Onlab\VOC_dataset\VOCdevkit\VOC2012\test\2007_000121.jpg: 384x512 2 tvmonitors, 4.0ms
image 10/22 c:\Onlab\VOC_dataset\VOCdevkit\VOC2012\test\2007_000123.jpg: 384x512 1 train, 3.0ms
image 11/

In [ ]:
from PIL import Image
import os
import matplotlib.pyplot as plt

# List of models
models = ['trainAdeamWopt+patience30', 'trainAdamWopt', 'trainautoopt', 'traindefhyppar']

# Base directory path
base_directory_path = 'c:\\Onlab\\VOC_dataset\\VOCdevkit\\VOC2012'

# Iterate through each model to get image files
for model_name in models:
    # Create the directory path
    directory_path = f'{base_directory_path}\\{model_name}'
    
    # Get all jpg files in the directory
    image_files = [f for f in os.listdir(directory_path) if f.endswith('.jpg')]
    
    # Iterate through each image file
    for image_file in image_files:
        # Create a figure and a grid of subplots with a larger size
        fig, axs = plt.subplots(1, len(models), figsize=(20, 20))
        
        # Iterate through each model again to display the same image from each model
        for i, model_name in enumerate(models):
            # Create the directory path for the current model
            current_directory_path = f'{base_directory_path}\\{model_name}'
            
            # Open the image file
            img = Image.open(f'{current_directory_path}\\{image_file}')
            
            # Display the image in a subplot
            axs[i].imshow(img)
            axs[i].set_title(model_name)
            axs[i].axis('off')
        
        # Display the figure with the images
        plt.show()